In [ ]:
# 회귀 데이터 전처리

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy import stats
from sklearn.datasets import load_boston
from sklearn.preprocessing import PolynomialFeatures
%matplotlib inline

boston = load_boston()

bostonDF = pd.DataFrame(boston.data, columns=boston.feature_names)

bostonDF['PRICE'] = boston.target

# 2개의 행과 4개의 열을 가진 subplots를 이용. axs는 4x2개의 ax를 가짐
# fig, axs = plt.subplots(figsize=(16, 8), ncols=4, nrows=2)
lm_features = ['RM', 'ZN', 'INDUS', 'NOX', 'AGE', 'PTRATIO', 'LSTAT', 'RAD']
'''for i, feature in enumerate(lm_features):
    row = int(i/4)
    col = i%4
    # seaborn의 regplot을 이용해 산점도와 선형회귀직선을 함께 표현
    sns.regplot(x=feature, y='PRICE', data=bostonDF, ax=axs[row][col])'''
    
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

y_target = bostonDF['PRICE']
X_data = bostonDF.drop(['PRICE'], axis=1, inplace=False)

X_train, X_test, y_train, y_test = train_test_split(X_data, y_target, test_size=0.3, random_state=156)

#선형회귀 OLS로 학습/예측/평가 수행
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

from sklearn.model_selection import cross_val_score

y_target = bostonDF['PRICE']
X_data = bostonDF.drop(['PRICE'], axis=1, inplace=False)

#cross_val_score()로 5폴드 세트로 MSE를 구한 뒤 이를 기반으로 다시 RMSE를 구함
neg_mse_scores = cross_val_score(lr, X_data, y_target, scoring='neg_mean_squared_error', cv=5)
rmse_scores = np.sqrt(-1 * neg_mse_scores)
avg_rmse = np.mean(rmse_scores)

# cross_val_score(scoring='neg_mean_squared_error')로 반환된 값은 모두 음수
print('5 folds의 개별 Negative MSE scores : ', np.round(neg_mse_scores, 2))
print('5 folds의 개별 RMSE scores : ', np.round(rmse_scores, 2))
print('5 folds의 평균 RMSE : {0:.3f}'.format(avg_rmse))

5 folds의 개별 Negative MSE scores :  [-12.46 -26.05 -33.07 -80.76 -33.31]
5 folds의 개별 RMSE scores :  [3.53 5.1  5.75 8.99 5.77]
5 folds의 평균 RMSE : 5.829


In [21]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet

# alpha 값에 따른 회귀 모델의 폴드 평균 RMSE를 출력하고 회귀 계수값들을 DataFrame으로 반환
def get_linear_reg_eval(model_name, params=None, X_data_n=None, y_target_n=None, verbose=True, return_coeff=True):
    coeff_df = pd.DataFrame()
    if verbose : print('##### ', model_name, '#####')
    for param in params:
        if model_name == 'Ridge': model = Ridge(alpha=param)
        elif model_name == 'Lasso' : model = Lasso(alpha=param)
        elif model_name == 'ElasticNet' : model = ElasticNet(alpha=param, l1_ratio=0.7)
        neg_mse_scores = cross_val_score(model, X_data_n, y_target_n, scoring='neg_mean_squared_error', cv=5)
        avg_rmse = np.mean(np.sqrt(-1*neg_mse_scores))
        print('alpha {0}일 때 5폴드 세트의 평균 RMSE : {1:.3f}'.format(param, avg_rmse))
        
        # cross_val_score는 evaluation metric만 반환하므로 모델을 다시 학습하여 회귀 계수 추출
        model.fit(X_data_n, y_target_n)
        if return_coeff:
            # alpha에 따른 피쳐별 회귀 계수를 series로 변환하고 이를 dataframe의 컬럼으로 추가
            coeff = pd.Series(data=model.coef_, index=X_data_n.columns)
            colname = 'alpha:' + str(param)
            coeff_df[colname] = coeff
            
    return coeff_df

In [24]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import numpy as np

# method는 표준 졍규 분포 변환(standard), 최댓값/최솟값 정규화(MinMax), 로그변환(log) 결정
# p_degree는 다항식 특성을 추가할 때 적용. p_degree는 2이상 부여하지 않음
def get_scaled_data(method='None', p_degree=None, input_data=None):
    if method == 'Standard':
        scaled_data =StandardScaler().fit_transform(input_data)
    elif method == 'MinMax':
        scaled_data = MinMaxScaler().fit_transform(input_data)
    elif method == 'Log':
        scaled_data = np.log1p(input_data)
    else:
        scaled_data = input_data
    
    if p_degree != None:
        scaled_data = PolynomialFeatures(degree=p_degree, include_bias=False).fit_transform(scaled_data)
        
    return scaled_data    

In [25]:
# Ridge의 alpha값을 다르게 적용하고 다양한 데이터 변환 방법에 따른 RMSE 추출
alphas = [0.1, 1, 10, 100]

# 5개 방식으로 변환, 먼저 원본 그대로, 표준정규분포, 표준정규분포+다항식 특성
# 최대/최소 정규화, 최대/최소 정규화+다항식 특성, 로그변환
scale_methods = [(None, None), ('Standard', None), ('Standard', 2), ('MinMax', None), ('MinMax', 2), ('Log', None)]

for scale_method in scale_methods:
    X_data_scaled = get_scaled_data(method=scale_method[0], p_degree=scale_method[1], input_data=X_data)
    print('\n## 변환 유형 : {0}, Polynomial Degree : {1}'.format(scale_method[0], scale_method[1]))
    get_linear_reg_eval('Ridge', params=alphas, X_data_n=X_data, y_target_n=y_target, verbose=False, return_coeff=False)


## 변환 유형 : None, Polynomial Degree : None
alpha 0.1일 때 5폴드 세트의 평균 RMSE : 5.788
alpha 1일 때 5폴드 세트의 평균 RMSE : 5.653
alpha 10일 때 5폴드 세트의 평균 RMSE : 5.518
alpha 100일 때 5폴드 세트의 평균 RMSE : 5.330

## 변환 유형 : Standard, Polynomial Degree : None
alpha 0.1일 때 5폴드 세트의 평균 RMSE : 5.788
alpha 1일 때 5폴드 세트의 평균 RMSE : 5.653
alpha 10일 때 5폴드 세트의 평균 RMSE : 5.518
alpha 100일 때 5폴드 세트의 평균 RMSE : 5.330

## 변환 유형 : Standard, Polynomial Degree : 2
alpha 0.1일 때 5폴드 세트의 평균 RMSE : 5.788
alpha 1일 때 5폴드 세트의 평균 RMSE : 5.653
alpha 10일 때 5폴드 세트의 평균 RMSE : 5.518
alpha 100일 때 5폴드 세트의 평균 RMSE : 5.330

## 변환 유형 : MinMax, Polynomial Degree : None
alpha 0.1일 때 5폴드 세트의 평균 RMSE : 5.788
alpha 1일 때 5폴드 세트의 평균 RMSE : 5.653
alpha 10일 때 5폴드 세트의 평균 RMSE : 5.518
alpha 100일 때 5폴드 세트의 평균 RMSE : 5.330

## 변환 유형 : MinMax, Polynomial Degree : 2
alpha 0.1일 때 5폴드 세트의 평균 RMSE : 5.788
alpha 1일 때 5폴드 세트의 평균 RMSE : 5.653
alpha 10일 때 5폴드 세트의 평균 RMSE : 5.518
alpha 100일 때 5폴드 세트의 평균 RMSE : 5.330

## 변환 유형 : Log, Polynomial Degree : None
alpha 0.1일 때